In [2]:
# computational approach (no ace)

import random
from tabulate import tabulate

RUN_AMT = 100000

def add_card():
    return random.randint(2, 10)

def play_bj(p, e, strat):
    if strat == "H":
        p += add_card()
        if p > 21:
            return -1
    while e < 17:
        e += add_card()
        if e > 21:
            return 1
    if p > e:
        return 1
    elif p < e:
        return -1
    else:
        return 1

# Create a dictionary to store the basic strategy chart
# Initialize with placeholders
strategy_chart = {}

# Start with the highest player total
for player_total in range(21, 11, -1):  # Player's total can be from 21 down to 12
    for dealer_upcard in range(2, 11):  # Dealer's upcard can be from 2 to 10
        hit_count = 0
        for _ in range(RUN_AMT):
            # Simulate player's decision to hit or stand (randomly)
            player_strategy = "H" if _ % 2 == 0 else "S" # half tests hit 1 card half tests stand
            
            # Simulate the game based on the chosen strategy
            result = play_bj(player_total, dealer_upcard, player_strategy)
            # Evaluate the result
            if result == 1 and player_strategy == "S" or result == -1 and player_strategy == "H":
                hit_count -= 1  
            else:
                hit_count += 1
        #print(player_total, dealer_upcard, hit_count, stand_count,a,b,c,d, a+c, b+d)
        # Determine the strategy based on the win percentage
        print(player_total, dealer_upcard, hit_count)
        if hit_count > 0:
            strategy_chart[(player_total, dealer_upcard)] = 'H'
        else:
            strategy_chart[(player_total, dealer_upcard)] = 'S'

# Create headers for the columns (dealer upcard)
column_headers = [str(upcard) for upcard in range(2, 11)]

# Create rows (player hard total) and fill in the table
table_data = []
for player_total in range(21, 11, -1):
    row = [strategy_chart[(player_total, dealer_upcard)] for dealer_upcard in range(2, 11)]
    table_data.append([player_total] + row)

# Print the table
table = tabulate(table_data, headers=["Player Total"] + column_headers, tablefmt="grid")

print(table)


21 2 -100000
21 3 -100000
21 4 -100000
21 5 -100000
21 6 -100000
21 7 -100000
21 8 -100000
21 9 -100000
21 10 -100000
20 2 -88636
20 3 -88254
20 4 -88368
20 5 -88410
20 6 -87968
20 7 -89268
20 8 -90492
20 9 -91220
20 10 -92892
19 2 -64372
19 3 -64174
19 4 -63758
19 5 -64546
19 6 -64098
19 7 -66554
19 8 -68628
19 9 -70856
19 10 -64160
18 2 -39124
18 3 -38804
18 4 -39358
18 5 -39540
18 6 -39742
18 7 -43148
18 8 -48004
18 9 -41246
18 10 -35498
17 2 -14120
17 3 -13832
17 4 -15268
17 5 -15578
17 6 -16236
17 7 -22276
17 8 -17018
17 9 -12410
17 10 -8858
16 2 9404
16 3 8920
16 4 7474
16 5 6664
16 6 3888
16 7 6662
16 8 10892
16 9 12696
16 10 13976
15 2 13726
15 3 13618
15 4 12704
15 5 11276
15 6 7902
15 7 12750
15 8 15882
15 9 17334
15 10 18170
14 2 17142
14 3 16466
14 4 15790
14 5 14618
14 6 11886
14 7 16268
14 8 18904
14 9 20666
14 10 20554
13 2 20090
13 3 19516
13 4 18714
13 5 17984
13 6 16294
13 7 19400
13 8 21526
13 9 23514
13 10 23672
12 2 23240
12 3 22580
12 4 21856
12 5 21162
12 6 19762

In [54]:
# from dataset (blkjckhands.csv) - 900,000 real blackjack hands
# https://www.kaggle.com/datasets/mojocolors/900000-hands-of-blackjack-results/data

import pandas as pd
import numpy as np

# Read data from the CSV file
data = pd.read_csv("blkjckhands.csv")

# Convert dealer card to string and handle 1 or 11 as Ace
data['dealcard1'] = data['dealcard1'].astype(str).replace({'1': 'A', '11': 'A'})

# Calculate player total
data['player_total'] = data['card1'].astype(int) + data['card2'].astype(int)

# Check for Ace
data['ace_card1'] = (data['card1'] == 1) | (data['card1'] == 11)
data['ace_card2'] = (data['card2'] == 1) | (data['card2'] == 11)

# Update player_total column with Ace handling
data['player_total'] = np.where(data['ace_card1'] & data['ace_card2'], 'AA',
                                np.where(data['ace_card1'], 'A' + data['card2'].astype(str),
                                         np.where(data['ace_card2'], 'A' + data['card1'].astype(str),
                                                  data['player_total'].astype(str))))

# Convert player_total to categorical with a custom sort order
custom_order = [str(i) for i in range(5, 21)] + [f'A{j}' for j in range(2, 11)] + ['AA']
data['player_total'] = pd.Categorical(data['player_total'], categories=custom_order, ordered=True)

# Drop rows with NaN values in player_total or dealcard1
data = data.dropna(subset=['player_total', 'dealcard1'])

# Initialize a matrix with rows for player_totals and columns for dealer cards
matrix_rows = data['player_total'].cat.categories
matrix_columns = [str(i) for i in range(2, 11)] + ['A']

# Create an empty matrix
matrix = pd.DataFrame(index=matrix_rows, columns=matrix_columns, dtype=int)
matrix = matrix.fillna(0)

# Define conditions
condition_1 = (data['card3'] == 0) & (data['winloss'] == 'Loss') | (data['card3'] > 0) & (data['winloss'] == 'Win')
condition_2 = (data['card3'] == 0) & (data['winloss'] == 'Win') | (data['card3'] > 0) & (data['winloss'] == 'Loss')

# Loop through rows and update matrix values
for _, row in data.iterrows():
    if condition_1.loc[_]:
        matrix.loc[row['player_total'], row['dealcard1']] += 1
    elif condition_2.loc[_]:
        matrix.loc[row['player_total'], row['dealcard1']] -= 1

# Replace positive values with 'H' and others with 'S'
result_matrix = np.where(matrix > 0, 'H', 'S')

# Create a new DataFrame with the updated values
result_df = pd.DataFrame(index=matrix_rows, columns=matrix_columns, data=result_matrix)

# Print the updated matrix
print(result_df)


     2  3  4  5  6  7  8  9 10  A
5    S  S  S  S  S  S  S  S  S  S
6    S  S  S  S  S  S  S  S  S  S
7    S  S  S  S  S  S  S  S  S  S
8    H  H  H  H  H  H  S  S  S  S
9    H  H  H  H  H  H  H  S  S  S
10   H  H  H  H  H  H  H  H  S  S
11   H  H  H  H  H  H  H  H  H  S
12   H  H  H  H  H  H  H  H  H  H
13   H  H  H  H  H  H  H  H  H  H
14   H  H  H  H  H  H  H  H  H  H
15   H  H  H  H  H  H  H  H  H  H
16   H  H  H  H  H  H  H  H  H  H
17   H  H  H  H  S  H  H  H  H  H
18   S  S  S  S  S  S  S  H  H  H
19   S  S  S  S  S  S  S  S  H  H
20   S  S  S  S  S  S  S  S  S  S
A2   S  H  S  H  H  H  S  S  S  S
A3   H  S  H  H  H  H  S  S  S  S
A4   S  H  H  H  H  S  S  S  S  S
A5   H  S  H  H  H  S  S  S  S  S
A6   H  H  H  H  H  H  S  S  S  S
A7   S  S  S  S  S  S  S  H  H  H
A8   S  S  S  S  S  S  S  S  H  H
A9   S  S  S  S  S  S  S  S  S  S
A10  S  S  S  S  S  S  S  S  S  S
AA   S  H  H  H  H  H  S  S  S  S


In [56]:
print(matrix)

          2       3       4       5       6       7       8       9       10  \
5    -120.0   -95.0   -98.0   -47.0   -22.0  -141.0  -176.0  -310.0  -1371.0   
6    -154.0  -107.0  -152.0  -100.0   -25.0  -189.0  -283.0  -487.0  -2065.0   
7    -202.0   -98.0  -113.0   -39.0    -5.0   -58.0  -324.0  -426.0  -2570.0   
8      82.0    29.0    88.0   220.0   180.0   215.0  -169.0  -453.0  -2544.0   
9     311.0   302.0   291.0   468.0   421.0   409.0   247.0  -126.0  -2052.0   
10    636.0   663.0   655.0   807.0   794.0   786.0   590.0   270.0   -404.0   
11    851.0   931.0   917.0  1054.0  1095.0   888.0   733.0   526.0    834.0   
12   1417.0  1465.0  1323.0  1162.0   920.0  1326.0  1619.0  2253.0  10392.0   
13   1759.0  1376.0  1253.0   959.0  1131.0  1570.0  1900.0  2214.0  10492.0   
14   1495.0  1224.0  1112.0   825.0   797.0  1770.0  1999.0  2308.0  10573.0   
15   1527.0  1203.0  1054.0   827.0   708.0  1749.0  1944.0  2315.0  10493.0   
16   1127.0   998.0   918.0   690.0   68

In [58]:
rows_with_ply2cardsum_5 = data[data['ply2cardsum'] == 5]

# Print the resulting rows
print(rows_with_ply2cardsum_5)

        Unnamed: 0 PlayerNo  card1  card2  card3  card4  card5  sumofcards  \
49               1  Player2      3      2      5     10      0          20   
56               2  Player3      1      4      7     10      0          22   
252              0  Player1      4      1      7     10      0          22   
352              4  Player5      1      4     10      7      0          22   
367              1  Player2      4      1     10      0      0          15   
...            ...      ...    ...    ...    ...    ...    ...         ...   
899685           3  Player4      3      2     10      0      0          15   
899807           5  Player6      2      3     10      0      0          15   
899887           1  Player2      3      2      8      3     10          26   
899930           2  Player3      3      2      4      2      1          12   
899975           5  Player6      2      3      8      0      0          13   

       dealcard1  dealcard2  ...  blkjck  winloss  plybustbeat 